In [10]:

import json
import pandas as pd
import requests
import datetime
import sqlalchemy


def get_weather(citys):
  
    today = datetime.datetime.now().date()
    next_day = today + datetime.timedelta(days=1)
    after_next_day = today + datetime.timedelta(days=2)

    weth= requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={citys}&appid=a9ea4aa6f8c466e40e9302eebafe6a90&units=metric")

    weth_json = weth.json()

    weth_info = weth_json['list']
    weth_info
    weth_info= pd.json_normalize(weth_info)
    weth_info
    weather = weth_info[['pop', 'dt_txt', 'main.feels_like', 'wind.speed']].copy()

    weather.columns = ['pop', 'time_date', 'perceived_temp', 'wind_speed']
    weather['city_name'] = citys
    weather["time_date"]= pd.to_datetime(weather["time_date"])

    weather = weather.loc[weather['time_date'] > str(next_day)]
    weather = weather.loc[weather['time_date'] < str(after_next_day)]

   

    return weather


def get_arrivals(icao):
    
    tomorrow = datetime.datetime.now().date() + datetime.timedelta(days=1)

    url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/"+icao+"/"+str(tomorrow)+"T08:00/"+str(tomorrow)+"T20:00"
    
    querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}

   
        
    
    
    headers = {
        "X-RapidAPI-Key": "35c1817acfmsh82cbf727450fa81p16bde7jsn54a382b5ba4f",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

    
   

    response = requests.request("GET", url, headers=headers, params=querystring)
    response = response.json()
    
    arrivals = response['arrivals']

    arrivals = pd.json_normalize(arrivals)
   
    try:
        
        arrivals = arrivals[['departure.airport.name','arrival.scheduledTimeUtc', 'arrival.terminal']].copy()
    except Exception: 
        arrivals = arrivals[['departure.airport.name','arrival.scheduledTimeUtc']].copy()
        arrivals['arrival.terminal']=-1

    arrivals.columns = ['departure_airport','arrival_time_utc', 'terminal']
    arrivals['airport_icao'] = icao
    
    arrivals['arrival_time_utc']= pd.to_datetime(arrivals['arrival_time_utc'])
    
    return arrivals


def lambda_handler(event, context):
    
    schema="AWS_Scooters"
    host="leandrowbspt.c4ugahw5lqgh.eu-central-1.rds.amazonaws.com"
    user="admin"
    password="ssq903nother"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    #FOR LOOP FOR WEATHER
    
    city_names = ['berlin', 'barcelona','zurich', 'malaga']
    cities = pd.read_sql('SELECT * FROM cities', con)
    get_weather

    results_weather = []

    for index, row in cities.iterrows():
        weather = get_weather(row ['city_name'])
        
        weather['city_id'] = row['city_id']
        
        weather.to_sql('weathers',con=con,if_exists='append',index=False)
        
        
        results_weather.append(weather)
   
    #FOR LOOP FOR ARRIVALS
    
    icao_names = ['LEBL', 'EDDB', 'LSZH', 'LEMG']

    results = []
    pd.read_sql('SELECT * FROM flights', con)
    
    for icao in icao_names:
        arrivals = get_arrivals(icao)
        results.append(arrivals)
        
        arrivals.to_sql('flights',con=con,if_exists='append',index=False)


    results = pd.concat(results)

    results


    


  
    
    
 

In [11]:
lambda_handler(0, 21)